# SCF optimization

In [1]:
import numpy as np
import veloxchem as vlx
import scipy
import matplotlib.pyplot as plt

mol_str = """3

O    0.000000000000        0.000000000000        0.000000000000
H    0.000000000000        0.740848095288        0.582094932012
H    0.000000000000       -0.740848095288        0.582094932012
"""

molecule = vlx.Molecule.from_xyz_string(mol_str)
basis = vlx.MolecularBasis.read(molecule, "cc-pVDZ")

* Warning * Environment variable OMP_NUM_THREADS not set.
* Warning * Setting OMP_NUM_THREADS to 8.
* Info * Reading basis set from file: /opt/anaconda3/envs/echem/lib/python3.9/site-packages/veloxchem/basis/CC-PVDZ       
                                                                                                                          
                                              Molecular Basis (Atomic Basis)                                              
                                                                                                                          
                               Basis: CC-PVDZ                                                                             
                                                                                                                          
                               Atom Contracted GTOs           Primitive GTOs                                              
                                       

In [2]:
norb = basis.get_dimensions_of_basis(molecule)
nocc = molecule.number_of_electrons() // 2
V_nuc = molecule.nuclear_repulsion_energy()

print("Number of contracted basis functions:", norb)
print("Number of doubly occupied molecular orbitals:", nocc)
print(f"Nuclear repulsion energy (in a.u.): {V_nuc : 14.12f}")

Number of contracted basis functions: 24
Number of doubly occupied molecular orbitals: 5
Nuclear repulsion energy (in a.u.):  9.343638157670


## Compute the integrals

In [4]:
# overlap matrix
overlap_drv = vlx.OverlapIntegralsDriver()
S = overlap_drv.compute(molecule, basis).to_numpy()

# one-electron Hamiltonian
kinetic_drv = vlx.KineticEnergyIntegralsDriver()
T = kinetic_drv.compute(molecule, basis).to_numpy()

nucpot_drv = vlx.NuclearPotentialIntegralsDriver()
V = -nucpot_drv.compute(molecule, basis).to_numpy()

h = T + V

# two-electron Hamiltonian
eri_drv = vlx.ElectronRepulsionIntegralsDriver()
g = np.zeros((norb, norb, norb, norb))
eri_drv.compute_in_memory(molecule, basis, g)

## Useful equations:

The HF density matrix is made from the occupied orbitals
$$D_{\mu \nu} = \sum_i^\mathrm{occ} C_\mu^i C_\nu^i = C_\mathrm{occ} C_\mathrm{occ}^T $$

The Fock matrix is a sum of one-electron, Coulomb and Exchange terms:

$$ F_{\mu\nu} = h_{\mu\nu} + \sum_{\lambda\sigma} \left( 2 g_{\mu\nu\lambda\sigma} D_{\lambda\sigma} - g_{\mu\sigma\lambda\nu} D_{\lambda\sigma}\right) $$

To optimize the orbitals, we solve the Roothan-Hall equation:

$$ \mathbf{F C} = \mathbf{S C} \epsilon $$

The energy is:

$$ E = 2 \sum_{\mu \nu} h_{\mu\nu} D_{\mu\nu} + \sum_{\mu\nu \lambda\sigma} \left( 2 g_{\mu\nu\lambda\sigma} D_{\lambda\sigma} - g_{\mu\sigma\lambda\nu}\right) D_{\mu\nu} D_{\lambda\sigma} $$

$$ E = \sum_{\mu \nu} \left( h_{\mu\nu} + F_{\mu\nu} \right) D_{\mu\nu}  $$

## Implementation

In [5]:
max_iter = 50
conv_thresh = 1e-4

# initial guess from core Hamiltonian
# Solve the generalized eigenvalue problem HC = SCE
eps, C = scipy.linalg.eigh(h, S)

print("iter      SCF energy    Error norm")

for iter in range(max_iter):

    # Form the density matrix
    Cocc = C[:, :nocc]
    D = Cocc @ Cocc.T

    # Form Fock matrix
    J = np.einsum("ijkl,kl->ij", g, D)
    K = np.einsum("ilkj,kl->ij", g, D)
    F = h + 2 * J - K

    # Trace h + F with the density matrix to get the energy
    E = np.sum((h+F)*D) + V_nuc

    # compute convergence metric
    Cvirt = C[:, nocc:]
    e_mat = Cocc.T @ F @ Cvirt #Occupied-virtual block of Fock matrix in MO
    error = np.linalg.norm(e_mat)

    print(f"{iter+1:>2d}  {E:16.8f}  {error:10.2e}")

    if error < conv_thresh:
        print("SCF iterations converged!")
        break

    #Solve the generalized eigenvalue problem FC = SCE
    eps, C = scipy.linalg.eigh(F, S)

iter      SCF energy    Error norm
 1      -68.84975229    2.23e+00
 2      -69.95937641    1.79e+00
 3      -73.34743276    1.74e+00
 4      -73.46688910    1.36e+00
 5      -74.74058933    1.29e+00
 6      -75.55859127    7.91e-01
 7      -75.86908635    4.86e-01
 8      -75.97444165    2.74e-01
 9      -76.00992921    1.60e-01
10      -76.02143957    8.99e-02
11      -76.02519173    5.15e-02
12      -76.02640379    2.92e-02
13      -76.02679653    1.67e-02
14      -76.02692347    9.45e-03
15      -76.02696455    5.38e-03
16      -76.02697784    3.06e-03
17      -76.02698213    1.74e-03
18      -76.02698352    9.89e-04
19      -76.02698397    5.63e-04
20      -76.02698412    3.20e-04
21      -76.02698416    1.82e-04
22      -76.02698418    1.04e-04
23      -76.02698418    5.89e-05
SCF iterations converged!


In [6]:
scf_drv = vlx.ScfRestrictedDriver()
scf_results = scf_drv.compute(molecule, basis)

                                                                                                                          
                                            Self Consistent Field Driver Setup                                            
                                                                                                                          
                   Wave Function Model             : Spin-Restricted Hartree-Fock                                         
                   Initial Guess Model             : Superposition of Atomic Densities                                    
                   Convergence Accelerator         : Two Level Direct Inversion of Iterative Subspace                     
                   Max. Number of Iterations       : 50                                                                   
                   Max. Number of Error Vectors    : 10                                                                   
                

# CASSCF calculation

In [ ]:
import multipsi as mtp

mol_str = """12

C         -3.80267        2.27360        0.00000
C         -3.83708        0.87920       -0.00000
C         -2.57787        2.94100       -0.00000
C         -1.38749        2.21400       -0.00000
C         -1.42190        0.81960       -0.00000
C         -2.64670        0.15220       -0.00000
H         -4.73011        2.84002        0.00000
H         -4.79133        0.35921        0.00000
H         -2.67350       -0.93420       -0.00000
H         -0.49446        0.25318       -0.00000
H         -2.55107        4.02740       -0.00000
H         -0.43324        2.73398       -0.00000
"""

molecule = vlx.Molecule.from_xyz_string(mol_str)
basis = vlx.MolecularBasis.read(molecule, "def2-sv(p)")

In [ ]:
viewer = mtp.OrbitalViewer()
viewer.plot_guess(molecule, basis)

In [ ]:
space = mtp.OrbSpace(molecule, "input-cas.h5")

mcscfdrv = mtp.McscfDriver()
mcscfdrv.compute(molecule, basis, space)

In [ ]:
viewer = mtp.OrbitalViewer()
viewer.plot(molecule, basis, space)